#### LCEL - LangChain Expression Language

In [5]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

#### Google LLM - Configure Model

In [1]:
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

/home/jatinder/Jatinder/LLMLearning/langchain-tutorials/LearningDocumentation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### The Pipe Operator: | Simple Chain with String Output using ChatPrmoptTemplate

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "bears"})

'Why did the bear say no to dessert?\n\nBecause he was stuffed! 🐻😂 \n'

#### The Pipe Operator: | Simple Chain with JSON Output using ChatPrmoptTemplate

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic} and give output in JSON only")

jsonOutputParser = JsonOutputParser()

json_chain = prompt | model | JsonOutputParser()

json_chain.invoke({"topic": "elephant and ant"})

{'joke': "Why did the elephant get lost in the ant farm? Because he couldn't find his way out of the anthill!"}

#### Coercion

###### We can even combine this chain with more runnables to create another chain. This may involve some input/output formatting using other types of runnables, depending on the required inputs and outputs of the chain components.

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

joke_chain = joke_prompt | model | StrOutputParser()

topic = "elephant and donkey"

#Just to see the result
joke_outupt = joke_chain.invoke({"topic": topic})
print(joke_outupt)

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": joke_chain} | analysis_prompt | model | StrOutputParser()

composed_chain.invoke({"topic": topic})

Why did the elephant cross the road? 

To prove to the donkey that he could! 



'That\'s a pretty good joke! It\'s funny because it\'s unexpected and plays on the stereotype of a donkey being stubborn and an elephant being strong. \n\nHere\'s why it works:\n\n* **Unexpected Twist:**  The setup leads you to expect a standard "why did the chicken cross the road" type of answer, but the punchline is a playful jab at the donkey.\n* **Animal Stereotypes:** The joke uses the common perception of donkeys being stubborn and elephants being powerful. This makes the punchline funnier because it plays on those familiar ideas.\n* **Humor in the Absurd:** The idea of an elephant crossing a road just to prove a point to a donkey is inherently absurd, which contributes to the humor.\n\nOverall, it\'s a simple but effective joke that delivers a chuckle! \n'

#### Other way to use above code. Using Lamdba / inserting custom logic

###### However, keep in mind that using functions like this may interfere with operations like streaming.

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

joke_chain = joke_prompt | model | StrOutputParser()

topic = "elephant and ant"

#Just to see the result
joke_outupt = joke_chain.invoke({"topic": topic})
print(joke_outupt)

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_lamda_chain = joke_chain | (lambda input: {"joke": input}) | analysis_prompt | model | StrOutputParser()

composed_lamda_chain.invoke({"topic": topic})

Why did the elephant get lost in the ant farm? 

Because he couldn't find his way out of the anthill! 🐜🐘 



'That\'s a cute joke! It\'s funny because it plays on the contrast between the large elephant and the tiny ant, and the absurdity of the elephant having to prove anything to the ant.  It\'s a bit of a silly and unexpected answer, which makes it humorous. \n\nHowever, it\'s not a classic "laugh-out-loud" joke. It\'s more of a chuckle-worthy, lighthearted joke that relies on a bit of wordplay and the unexpectedness of the answer. \n'

## The .pipe() method

###### We could also compose the same sequence using the .pipe() method. Here's what that looks like:

In [18]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": joke_chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

'That\'s a decent joke! It\'s a play on words that uses the double meaning of "jump" – to quickly enter or exit something, and to travel through hyperspace. \n\nHere\'s why it works:\n\n* **Relatable:**  Most people can relate to the frustration of finding a good parking spot. \n* **Unexpected:** The joke uses a surprising twist by connecting a common issue (parking) with a sci-fi element (Battlestar Galactica).\n* **Clever:** The pun works because it highlights Starbuck\'s skill as a pilot, which is relevant to both the joke\'s setup and punchline.\n\n**Overall:**  It\'s a good, clean joke that will likely get a chuckle from fans of Battlestar Galactica. \n'